In [1]:
import sys
import torch
import os

In [2]:
if "gan.ipynb" in os.listdir():
    os.chdir("..")

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
from src.data_utils.batch import rebatch
from src.data_utils.data import get_training_iterators
from src.model.loss_optim import MultiGPULossCompute, SimpleLossCompute
from src.model.model import make_model, NoamOpt, LabelSmoothing, translate_sentence
from src.utils.utils import get_tokenizer

In [5]:
tok = get_tokenizer("tr")

In [6]:
train_iter, valid_iter, test_iter, train_idx, dev_idx, test_idx = get_training_iterators("tur", batch_size=512)

/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: MyIterator class will be retired soon and moved to torchtext.l

In [7]:
# mini dev set
with open("data/tr/tur.dev.tgt", encoding="utf-8") as infile:
    toystrings = [x.strip() for x in infile.readlines()[:20]]

In [8]:
toyset = [torch.LongTensor([1] + tok.Encode(x) + [2])  for x in toystrings]
toyset = torch.nn.utils.rnn.pad_sequence(sequences=toyset, padding_value=3)

In [9]:
toyset

tensor([[    1,     1,     1,  ...,     1,     1,     1],
        [ 5605,     8,  1330,  ...,     8,   771,  2804],
        [27861,  2475, 10284,  ...,  3987,  5057, 11694],
        ...,
        [    3,     3,     3,  ...,     3,     3,     3],
        [    3,     3,     3,  ...,     3,     3,     3],
        [    3,     3,     3,  ...,     3,     3,     3]])

Two critics:
- Input related to output or not
- Classifier into poetry, prose, generated, scrambled poetry

One word/token selector:
- Choose tokens from input sequence to use for topic
- 

In [10]:
from torchtext import data
import torchtext as tt
from src.data_utils.batch import MyIterator
from src.model.model import batch_size_val

def each_line(fname):
    c = 0
    lines = []
    with open(fname, "r", encoding="utf-8") as infile:
        for line in infile:
            if line.count(" ") > 200 or line.count(" ") < 10:
                continue
            lines.append(line.strip())
            c += 1
            if c >= 2000000: 
                break
    return lines

def make_iter(lines, tokenizer, batch_size=256):
    
    def tok(seq):
        return tokenizer.EncodeAsIds(seq)

    field = data.Field(tokenize=tok, init_token=1, eos_token=2, pad_token=3, use_vocab=False)
    #ds = data.TabularDataset(fpath, "tsv", [("src", field)], skip_header=True)

    examples = [tt.data.Example.fromdict({"src": x}, {"src": ("src", field)}) for x in lines]
    ds = tt.data.Dataset(examples, {"src": field})
    iter = MyIterator(ds, batch_size=batch_size, device="cpu",
                             repeat=False, sort_key=lambda x: len(x.src),
                             batch_size_fn=batch_size_val, train=False, sort=True)

    return iter




In [11]:
prose_iter = make_iter(each_line("data/tr/prose/prose_gan.txt"), tok, batch_size=512)


/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [12]:
import random

to_scramble = each_line("data/tr/tur.train.tgt")
scrambled = []
for poem in to_scramble:
    new_poem = poem.split("¬")
    random.shuffle(new_poem)
    scrambled.append("¬".join(new_poem))

In [13]:
scrambled_iter = make_iter(scrambled, tok, batch_size=3000)

In [14]:
import copy
from src.model.model import MultiHeadedAttention, PositionwiseFeedForward, \
                    PositionalEncoding, Encoder, EncoderLayer, Generator, Embeddings
import torch.nn as nn

class Critic(nn.Module):

    def __init__(self, encoder, src_embed, generator):
        super(Critic, self).__init__()
        self.encoder = encoder
        self.src_embed = src_embed
        self.generator = generator
        self.steps = 0

        
    def forward(self, x, mask):
        """Pass the input (and mask) through each layer in turn."""
        x = self.src_embed(x)
        for layer in self.encoder.layers:
            x = layer(x, mask)
        return self.encoder.norm(x)    


def make_critic(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    """Helper: Construct a model from hyper-parameters."""
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    generator = Generator(d_model, tgt_vocab)
    embed = nn.Sequential(Embeddings(d_model, src_vocab), c(position))
    encoder = Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N)
    critic = Critic(encoder, embed, generator)
    
    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in critic.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)

    return critic


In [15]:
ntokens = 32000
enc_dec = make_model(ntokens, ntokens, N=6).to(device)
token_selector = make_critic(ntokens, 2, N=2).to(device)
style_critic = make_critic(ntokens, 4, N=2).to(device)
relevance_critic = make_critic(ntokens + 1, 1, N=2).to(device)

/auto/plzen1/home/memduh/versetorch/src/model/model.py:264: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [16]:
from torch.autograd import Variable
import numpy as np
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


def prep_tensors( src, trg, pad=3):
    src_mask = (src != pad).unsqueeze(-2)
    trg_in = trg[:, :-1]
    trg_y = trg[:, 1:]
    trg_mask = make_std_mask(trg_in, pad)
    return src, trg_y, src_mask, trg_mask

def make_std_mask(tgt, pad):
    """Create a mask to hide padding and future words."""
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(
        subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
    return tgt_mask


In [17]:
def get_dae_input(tgt, token_selector):
    select_prob_embeds = token_selector.forward(tgt.to(device), 
                                         (tgt != 3).unsqueeze(-2).to(device))
    select_prob = token_selector.generator(select_prob_embeds)
    select_indices = torch.max(select_prob, dim=2).indices.type(torch.ByteTensor)
    dae_list = []
    for ind, row in zip(select_indices, tgt):
        dae_list.append(torch.masked_select(row, ind)[:15])
    dae_input = torch.nn.utils.rnn.pad_sequence(dae_list, batch_first=False, padding_value=3)
    return dae_input

In [18]:
rebatched = (rebatch(3, b) for b in train_iter)

In [19]:
torch


<module 'torch' from '/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torch/__init__.py'>

In [20]:
from src.model.adafactor import Adafactor

enc_dec_opt = NoamOpt(enc_dec.src_embed[0].d_model, 1, 2000,
                        torch.optim.Adam(enc_dec.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
#enc_dec_opt = Adafactor(enc_dec.parameters())

style_criterion = nn.BCELoss()
relevance_criterion = nn.BCELoss()

token_optim = Adafactor(token_selector.parameters())
style_optim = Adafactor(style_critic.parameters())
rel_optim = Adafactor(relevance_critic.parameters())

In [21]:
relevance_criterion = torch.nn.BCEWithLogitsLoss()

In [22]:
def get_relevance_input(dae_input, tgt):
    mid_point = torch.ones((tgt.shape[0], 1), dtype=torch.long) * ntokens
    return torch.cat((dae_input, mid_point.to(device), tgt), dim=1)


In [23]:
accumulation_steps = 8

In [24]:
# get validation iterator


def validate_batch(model, src, max_len=256, start_symbol=1, end_symbol=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    src_mask = (src != 3).unsqueeze(-2)
    memory = model.encode(src.to(device), src_mask.to(device))
    ys = torch.ones(src.shape[0], 1).fill_(start_symbol).type_as(src.data).to(device)
    finished = torch.zeros((src.shape[0], 1))
    for i in range(max_len-1):
        out = model.decode(memory, src_mask,
                           Variable(ys).to(device),
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)).to(device))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        # next_word = next_word.data_utils[0]
        unsqueezed = next_word.unsqueeze(1)
        for c, token in enumerate(unsqueezed):
            if token == end_symbol:
                finished[c] = 1
        if sum(finished) >= src.shape[0]:
            break
        ys = torch.cat([ys, unsqueezed], dim=1)
                        # torch.ones(src.shape[0], 1).type_as(src.data_utils).fill_(next_word).to(device)], dim=1)
    return ys


def greedy_generate(model, src, max_len=256, start_symbol=1, end_symbol=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    src_mask = (src != 3).unsqueeze(-2)
    memory = model.encode(src.to(device), src_mask.to(device))
    ys = torch.ones(src.shape[0], 1).fill_(start_symbol).type_as(src.data).to(device)
    finished = torch.zeros((src.shape[0], 1))
    for i in range(max_len-1):
        out = model.decode(memory, src_mask,
                           Variable(ys).to(device),
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)).to(device))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        # next_word = next_word.data_utils[0]
        unsqueezed = next_word.unsqueeze(1)
        for c, token in enumerate(unsqueezed):
            if finished[c] == 1:
                unsqueezed[c] = 3
            if token == end_symbol:
                finished[c] = 1
        if sum(finished) >= src.shape[0]:
            break
        ys = torch.cat([ys, unsqueezed], dim=1)
                        # torch.ones(src.shape[0], 1).type_as(src.data_utils).fill_(next_word).to(device)], dim=1)
    return ys

In [25]:

label_smoothing = LabelSmoothing(size=32000, padding_idx=3, smoothing=0.1)

/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [26]:
[tok.Decode(x.tolist()) for x in toyset.transpose(0, 1)]

['ayrılığın sınırlı ışığında¬ışıldar gözler¬dar gelir dünya¬sıkıcı karanlıktan¬çıkış başlar¬sevgi akla gelince¬ışık küçücüktür¬sevgi karanlığında¬sonsuzluğu hatırlatır insana¬ışık kandırıcıdır belki¬ayrılığı sevenler için¬orada yuvarlak bir masa¬üç beş sigara biraz da çay¬sevgi karanlığında herşey¬coşkuyla¬zaten aranan da o değil mi¬sınırlı değilmiş hiçbir şey',
 'Şair sözün gösterer,¬Gözel gözün gösterer.¬Payız özün gösterer,¬Saralmış yarpağıyla.¬Uzaqda quzey durar,¬Şa ⁇ ta qelbi hey vurar.¬Önümde muzey qurar,¬Heyat çılpaqlığıyla.¬Yel eser deyinerek,¬ ⁇ um-paltar geyinerek,¬Fe ⁇ r eder öyünərek¬Insan torpaqlığıyla.',
 'Ey yakarış¬Ey uyanış¬Ey sevgi¬Ey aşk¬Ey arayış¬Ey tükeniş¬Ey özleyiş¬Uyanın artık¬O derin uykulardan¬Ve devam edin¬Katıksızca aramaya¬Aşkı,sevdayı,¬Yakarışı,uyanışı¬Arayışı,tükenişi¬Özleyişi,özlemi',
 'ben ölsemde aşkım cümle alemde¬kulakdan kulaga yayılır belki¬adımız gecer her bir kalemde¬gönül nikahımız kıyılır belki¬sevdik sevildik ölürcesine¬başını sonunu bilircesi

In [27]:
!ls checkpoints

cz-acc	     cz-dae   eng-baseline  tur-acc	  tur-dae
cz-baseline  eng-acc  eng-dae	    tur-baseline


In [28]:
last = sorted(os.listdir("checkpoints/tur-dae"), reverse=False, key=lambda x: int(x.partition(".")[0]))
hundred_k_plus = [x for x in last if len(x) > 8]


In [29]:
checkpoint = torch.load("checkpoints/tur-dae/" + hundred_k_plus[0])
token_selector.load_state_dict(checkpoint['selector_state_dict'])
token_optim.load_state_dict(checkpoint['selector_optim_state_dict'])
enc_dec.load_state_dict(checkpoint['model_state_dict'])
#enc_dec_opt.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

<All keys matched successfully>

In [ ]:
import time 

soft = torch.nn.Softmax(dim=1)
start = time.time()
all_tokens = 0
for c, (poetry_batch, prose_batch, scrambled_batch) in enumerate(zip(rebatched, prose_iter, scrambled_iter)):
    enc_dec_opt.optimizer.zero_grad()
    token_optim.zero_grad()
    all_tokens += poetry_batch.ntokens
    tgt, tgt_mask = poetry_batch.trg.to(device), poetry_batch.trg_mask.to(device)
    # classify tokens, get the first 15 tokens selected.
    dae_input = get_dae_input(poetry_batch.trg, token_selector).transpose(0,1).to(device)
    # create src and src mask from selected tokens
    dae_input_mask = (dae_input != 3).unsqueeze(-2)
    
    # get output of poetry generator
    #output_embeds = enc_dec.forward(dae_input, tgt, dae_input_mask, tgt_mask)
    #output = enc_dec.generator(output_embeds)
    #reconstruction_loss = label_smoothing(output.contiguous().view(-1, output.size(-1)),
    #                         poetry_batch.trg_y.to(device).contiguous().view(-1)) / poetry_batch.ntokens
    #reconstruction_loss.backward()
    
    # critic output
    output_selected = greedy_generate(enc_dec, dae_input)
    critic_scores = soft(style_critic.generator(style_critic.forward(output_selected.to(device), 
                                            (output_selected != 3).unsqueeze(-2).to(device)))[:,0,:])

    critic_loss = style_criterion(critic_scores[:, 0], torch.ones((critic_scores.shape[0])).to(device))
    critic_loss.backward()
    
    
    token_optim.step() 
    enc_dec_opt.step()
    
    # train critic (real, fake, scrambled, prose)
    style_optim.zero_grad()

    real_scores = soft(style_critic.generator(style_critic.forward(poetry_batch.trg_y.to(device), 
                                            (poetry_batch.trg_y != 3).unsqueeze(-2).to(device)))[:,0,:])
    crit_bce = style_criterion(real_scores[:, 0], torch.ones((tgt.shape[0])).to(device))
    
    gen_scores = soft(style_critic.generator(style_critic.forward(output_selected.to(device), 
                                            (output_selected != 3).unsqueeze(-2).to(device)))[:,0,:])
    crit_bce += style_criterion(gen_scores[:, 1], torch.ones((tgt.shape[0])).to(device))
    
    scramb_scores = soft(style_critic.generator(style_critic.forward(scrambled_batch.src.transpose(0,1).to(device),
                                            (scrambled_batch.src.transpose(0,1) != 3).unsqueeze(-2).to(device)))[:,0,:])
    crit_bce += style_criterion(scramb_scores[:, 2], torch.ones((scramb_scores.shape[0])).to(device))
    
    prose_scores = soft(style_critic.generator(style_critic.forward(prose_batch.src.transpose(0,1).to(device),
                            (prose_batch.src.transpose(0,1) != 3).unsqueeze(-2).to(device)))[:,0,:])
    crit_bce += style_criterion(prose_scores[:, 3], torch.ones((prose_scores.shape[0])).to(device))
    crit_bce.backward()
    style_optim.step()

    


    
    if c % 10  == 0:
        #print("Reconstruction loss:", reconstruction_loss)
        print("Discriminator loss:", critic_scores)
        print("Discriminator prediction on real poetry", real_scores)
        print("Discriminator prediction on scrambled", scramb_scores)
        print("Discriminator prediction on prose", prose_scores)
        print(all_tokens / (time.time() - start), "tokens processed per second.")
        #print(torch.mean(gen_scores[:, 1]))
        if c % 500 == 0:
            val_src = get_dae_input(toyset.transpose(0, 1), token_selector).transpose(0,1).to(device)
            validated = greedy_generate(enc_dec, val_src)
            print([tok.Decode(x.tolist()) for x in validated])
        

Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.


/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:733.)
  


Discriminator loss: tensor([[0.0828, 0.2911, 0.4840, 0.1421],
        [0.2397, 0.1369, 0.4136, 0.2098]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[0.4944, 0.0673, 0.3445, 0.0937],
        [0.7420, 0.0362, 0.1320, 0.0898]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on scrambled tensor([[0.5870, 0.1640, 0.0427, 0.2063],
        [0.7260, 0.1334, 0.0985, 0.0422],
        [0.8260, 0.1045, 0.0271, 0.0424],
        [0.6001, 0.2649, 0.0287, 0.1063],
        [0.6293, 0.0960, 0.1530, 0.1218],
        [0.5640, 0.2067, 0.1007, 0.1287],
        [0.8927, 0.0283, 0.0576, 0.0214],
        [0.6823, 0.0909, 0.1616, 0.0652],
        [0.6290, 0.0742, 0.1346, 0.1622],
        [0.4441, 0.1834, 0.3464, 0.0261],
        [0.8813, 0.0410, 0.0211, 0.0565],
        [0.7922, 0.1241, 0.0468, 0.0369],
        [0.6862, 0.1507, 0.1068, 0.0563],
        [0.8664, 0.0621, 0.0254, 0.0461],
        [0.3100, 0.0804, 0.5073, 0.1023],


['ayrılığın sınırlı ışığında¬ışıldar gözler¬dar gelir dünya¬sıkıcıyla¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de¬bir de', 'Şair sözün gösterer,¬Gözel gözün gösterer.¬Payızını ver de ver,¬Şairlik, şiir, şiir.¬Şair sözün göstersin,¬Gözel gözün göstersin,¬Payızını ver de ver,¬Şairlik, şiir, şiir.¬Şair sözün göstersin,¬Gözel gözün göstersin,¬Payızını ver de ver,¬Şairlik, şiir, şiir.¬Şair sözün göstersin,¬Gözel gözün göstersin,¬Payızını ver de ver,¬Şairlik, şiir, şiir.', 'Ey yakarış¬Ey uyan

Discriminator loss: tensor([[1.1806e-02, 9.8800e-01, 1.7730e-05, 1.7706e-04],
        [1.7671e-02, 9.8227e-01, 6.4784e-06, 4.9086e-05],
        [4.7167e-02, 9.5250e-01, 5.7659e-05, 2.7964e-04],
        [9.7555e-03, 9.9008e-01, 1.3133e-05, 1.4860e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[8.1556e-02, 9.1817e-01, 2.5171e-05, 2.5084e-04],
        [2.1096e-01, 7.8872e-01, 1.6285e-05, 2.9764e-04],
        [2.8932e-01, 7.1025e-01, 2.3878e-05, 4.0796e-04],
        [2.2424e-01, 7.7521e-01, 2.5314e-05, 5.3035e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on scrambled tensor([[3.6650e-03, 1.0571e-04, 9.8359e-01, 1.2641e-02],
        [4.0591e-03, 1.6920e-04, 9.9176e-01, 4.0095e-03],
        [1.9687e-03, 4.6551e-05, 9.9389e-01, 4.0977e-03],
        [3.8247e-03, 6.0476e-05, 9.9403e-01, 2.0888e-03],
        [8.8062e-04, 4.7247e-05, 9.9750e-01, 1.5745e-03],
        [2.2452e-03, 5.5609e-05, 9.9435e-01,

...

Discriminator loss: tensor([[4.7269e-06, 9.9998e-01, 3.5309e-07, 1.3602e-05],
        [1.4645e-06, 9.9999e-01, 3.5923e-07, 3.2764e-06],
        [2.0089e-07, 1.0000e+00, 1.9280e-06, 1.7012e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[1.0000e+00, 7.4186e-12, 9.3044e-12, 1.4971e-09],
        [1.0000e+00, 4.7642e-11, 6.6758e-12, 1.0112e-08],
        [1.0000e+00, 1.0024e-10, 6.1001e-12, 5.4647e-09]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on scrambled tensor([[4.9394e-07, 5.0346e-07, 9.9999e-01, 4.8297e-06],
        [5.9197e-08, 9.4650e-08, 1.0000e+00, 1.3271e-06],
        [2.7995e-08, 5.5543e-08, 1.0000e+00, 1.3369e-06],
        [5.9779e-08, 8.3317e-08, 1.0000e+00, 1.9869e-06],
        [4.3427e-08, 5.8876e-08, 1.0000e+00, 7.6465e-07],
        [1.2849e-07, 2.6227e-07, 1.0000e+00, 2.8429e-06],
        [5.4323e-08, 6.8787e-08, 1.0000e+00, 7.1887e-07],
        [1.9902e-07, 1.8888e-07, 1.0000e+00,

Discriminator loss: tensor([[5.2095e-07, 9.9999e-01, 1.8046e-06, 3.6355e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[1.0000e+00, 1.6103e-10, 4.4605e-11, 8.8813e-09]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on scrambled tensor([[8.4751e-06, 1.3468e-04, 9.9972e-01, 1.3399e-04],
        [8.9186e-06, 1.9929e-04, 9.9972e-01, 6.8610e-05],
        [1.1092e-05, 4.3977e-04, 9.9951e-01, 4.2837e-05],
        [2.0118e-05, 3.9566e-04, 9.9951e-01, 7.1567e-05],
        [9.2637e-06, 4.4976e-04, 9.9949e-01, 5.5932e-05],
        [2.4086e-05, 2.4586e-04, 9.9965e-01, 7.8611e-05],
        [1.6969e-05, 1.4872e-04, 9.9980e-01, 3.7534e-05],
        [5.4942e-06, 3.5548e-04, 9.9961e-01, 2.7844e-05],
        [1.9884e-05, 1.6892e-04, 9.9969e-01, 1.2142e-04],
        [1.6485e-05, 5.7311e-04, 9.9933e-01, 8.4492e-05],
        [1.1201e-05, 2.5575e-04, 9.9968e-01, 5.2174e-05],
        [3.6477e-05, 5.0739e-04, 9.9936e-01,

Discriminator loss: tensor([[1.1070e-06, 1.0000e+00, 3.2225e-07, 1.8137e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[1.0000e+00, 1.7630e-10, 5.1918e-11, 2.4248e-09]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on scrambled tensor([[1.1476e-06, 2.6608e-05, 9.9997e-01, 3.4912e-06],
        [7.9701e-06, 9.3104e-05, 9.9989e-01, 1.2094e-05],
        [4.7126e-06, 5.0973e-05, 9.9994e-01, 3.8268e-06],
        [1.6626e-05, 1.9080e-04, 9.9977e-01, 1.9525e-05],
        [1.6310e-06, 3.6919e-05, 9.9995e-01, 7.5080e-06],
        [1.5118e-05, 2.4007e-04, 9.9966e-01, 8.6256e-05],
        [2.3570e-06, 2.0429e-05, 9.9996e-01, 2.1814e-05],
        [1.3578e-05, 7.2134e-05, 9.9989e-01, 2.2338e-05],
        [5.7694e-06, 5.6295e-05, 9.9993e-01, 1.2578e-05],
        [1.8940e-05, 2.1279e-04, 9.9971e-01, 5.9244e-05],
        [1.3792e-06, 1.2921e-05, 9.9998e-01, 8.2097e-06],
        [2.6287e-07, 1.9577e-05, 9.9998e-01,

Discriminator loss: tensor([[6.5964e-06, 9.9999e-01, 1.0020e-06, 4.3576e-06],
        [1.2198e-07, 1.0000e+00, 8.5718e-07, 5.1309e-07],
        [7.5644e-07, 9.9999e-01, 1.1730e-05, 3.2769e-07],
        [1.3659e-07, 9.9997e-01, 2.7431e-05, 1.1061e-06],
        [2.7320e-06, 9.9999e-01, 5.2171e-06, 3.6604e-07],
        [5.4293e-07, 9.9999e-01, 5.5642e-06, 5.4437e-07],
        [9.7248e-06, 9.9995e-01, 3.7482e-05, 1.3198e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[1.0000e+00, 3.3162e-06, 2.2240e-07, 5.4766e-08],
        [1.0000e+00, 1.2546e-08, 2.0772e-08, 4.2853e-09],
        [1.0000e+00, 1.3654e-08, 2.8900e-09, 4.5572e-09],
        [1.0000e+00, 1.5778e-07, 7.2570e-08, 1.5378e-08],
        [1.0000e+00, 1.3705e-06, 1.7641e-07, 8.5396e-08],
        [1.0000e+00, 1.1220e-07, 5.4359e-08, 3.3153e-08],
        [1.0000e+00, 1.6533e-06, 3.3109e-08, 1.5325e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction

Discriminator loss: tensor([[3.4629e-07, 1.0000e+00, 2.7159e-06, 2.7547e-07],
        [1.3693e-05, 9.9975e-01, 2.3034e-04, 7.1837e-06],
        [8.5408e-08, 9.9999e-01, 5.8753e-06, 5.7797e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on real poetry tensor([[1.0000e+00, 3.5141e-11, 4.8570e-11, 3.9571e-09],
        [1.0000e+00, 1.4033e-11, 6.1986e-12, 1.6762e-09],
        [1.0000e+00, 7.6656e-11, 1.1023e-11, 2.1188e-09]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
Discriminator prediction on scrambled tensor([[5.5091e-07, 2.2387e-05, 9.9997e-01, 2.4822e-06],
        [1.3421e-06, 3.5875e-05, 9.9996e-01, 3.4647e-06],
        [1.4542e-06, 1.6555e-05, 9.9998e-01, 2.5591e-06],
        [3.0038e-07, 1.7039e-05, 9.9998e-01, 1.3524e-06],
        [5.3318e-07, 4.4128e-05, 9.9995e-01, 6.1301e-06],
        [1.3596e-06, 5.0304e-05, 9.9995e-01, 2.5272e-06],
        [1.0718e-06, 2.7573e-05, 9.9997e-01, 1.8713e-06],
        [2.4175e-06, 1.0826e-04, 9.9989e-01,